In [1]:
import pandas as pd, numpy as np

In [96]:
level_progress=pd.read_csv('level_progress.csv',parse_dates=True,index_col='event_datetime')
players=pd.read_csv('players.csv',parse_dates=True,index_col='install_datetime')

,player_id,platform,country,screen_size,system_memory
install_datetime,,,,,
2023-01-01 00:45:16.132,5EBDFF6D66C42698F132C01DC026573D,android,US,5.712000,3616
2022-12-19 00:59:46.194,A0B2352754DBC05198E6917F1C24A83F,ios,AU,6.054062,3851
2022-12-09 21:09:26.520,FE5DB1168E0C3A057CD6FFA0474599F5,android,US,5.316275,5627
2022-12-14 15:57:05.836,25008EB8E0F7E7694C412AE47F125B21,ios,US,6.054062,3851
2023-01-01 20:40:11.885,40798E31D33D6E4968BD8E916C116722,android,US,5.777590,7461
...,...,...,...,...,...
2022-12-15 23:45:09.983,4082186821C463F649853A5B512A4332,android,US,5.696880,10312
2022-12-16 00:59:57.637,22F6CB66F8E58622C89AC5ED651C9936,android,US,6.090847,2785
2022-12-22 00:55:19.064,42E11B30A562905866FF97C0BCDAA131,ios,US,6.055348,2831


In [98]:
data=(players.groupby(players['player_id']).count())
data[data['platform']==2]
players.drop(players[players['player_id']=='3D8A1EA896393A2FC953506BE9044428'].index[1],inplace=True)

In [116]:
players

,player_id,platform,country,screen_size,system_memory
install_datetime,,,,,
2023-01-01 00:45:16.132,5EBDFF6D66C42698F132C01DC026573D,android,US,5.712000,3616
2022-12-19 00:59:46.194,A0B2352754DBC05198E6917F1C24A83F,ios,AU,6.054062,3851
2022-12-09 21:09:26.520,FE5DB1168E0C3A057CD6FFA0474599F5,android,US,5.316275,5627
2022-12-14 15:57:05.836,25008EB8E0F7E7694C412AE47F125B21,ios,US,6.054062,3851
2023-01-01 20:40:11.885,40798E31D33D6E4968BD8E916C116722,android,US,5.777590,7461
...,...,...,...,...,...
2022-12-15 23:45:09.983,4082186821C463F649853A5B512A4332,android,US,5.696880,10312
2022-12-16 00:59:57.637,22F6CB66F8E58622C89AC5ED651C9936,android,US,6.090847,2785
2022-12-22 00:55:19.064,42E11B30A562905866FF97C0BCDAA131,ios,US,6.055348,2831


In [122]:
MasterDataset=level_progress.reset_index().merge(right=players.reset_index(),on='player_id')
MasterDataset

,event_datetime,player_id,level_number,stage_number,status,session_id,install_datetime,platform,country,screen_size,system_memory
0,2022-12-08 00:54:51.813,00030CDB447CCCE184BBF30B3F166EAB,1,1,start,FB4B79C03AF723057EE81F7416061D06,2022-12-08 00:54:37.555,ios,US,9.387676,5618
1,2022-12-08 00:56:52.981,00030CDB447CCCE184BBF30B3F166EAB,1,1,complete,BB5F69316E29360EA35533C8B6EAA6C0,2022-12-08 00:54:37.555,ios,US,9.387676,5618
2,2022-12-08 00:57:07.960,00030CDB447CCCE184BBF30B3F166EAB,1,2,start,BB5F69316E29360EA35533C8B6EAA6C0,2022-12-08 00:54:37.555,ios,US,9.387676,5618
3,2022-12-08 00:58:14.173,00030CDB447CCCE184BBF30B3F166EAB,1,2,complete,72D31206402CD71E19C68A7FB712E128,2022-12-08 00:54:37.555,ios,US,9.387676,5618
4,2022-12-08 00:58:27.386,00030CDB447CCCE184BBF30B3F166EAB,2,1,start,72D31206402CD71E19C68A7FB712E128,2022-12-08 00:54:37.555,ios,US,9.387676,5618
...,...,...,...,...,...,...,...,...,...,...,...
83277,2022-12-28 22:33:08.442,FFF8009A67AF2B8DF2749D9C11C7DC86,4,3,fail,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885
83278,2022-12-28 22:33:31.650,FFF8009A67AF2B8DF2749D9C11C7DC86,4,3,complete,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885
83279,2022-12-28 22:34:01.408,FFF8009A67AF2B8DF2749D9C11C7DC86,5,1,start,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885
83280,2022-12-28 22:34:51.820,FFF8009A67AF2B8DF2749D9C11C7DC86,5,1,complete,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885


In [144]:
MasterDataset['Phase']=(MasterDataset['level_number']-1)*3+MasterDataset['stage_number']
MasterDataset.drop(['level_number','stage_number'],axis=1,inplace=True)

In [149]:
MasterDataset

,event_datetime,player_id,status,session_id,install_datetime,platform,country,screen_size,system_memory,Phase
0,2022-12-08 00:54:51.813,00030CDB447CCCE184BBF30B3F166EAB,start,FB4B79C03AF723057EE81F7416061D06,2022-12-08 00:54:37.555,ios,US,9.387676,5618,1
1,2022-12-08 00:56:52.981,00030CDB447CCCE184BBF30B3F166EAB,complete,BB5F69316E29360EA35533C8B6EAA6C0,2022-12-08 00:54:37.555,ios,US,9.387676,5618,1
2,2022-12-08 00:57:07.960,00030CDB447CCCE184BBF30B3F166EAB,start,BB5F69316E29360EA35533C8B6EAA6C0,2022-12-08 00:54:37.555,ios,US,9.387676,5618,2
3,2022-12-08 00:58:14.173,00030CDB447CCCE184BBF30B3F166EAB,complete,72D31206402CD71E19C68A7FB712E128,2022-12-08 00:54:37.555,ios,US,9.387676,5618,2
4,2022-12-08 00:58:27.386,00030CDB447CCCE184BBF30B3F166EAB,start,72D31206402CD71E19C68A7FB712E128,2022-12-08 00:54:37.555,ios,US,9.387676,5618,4
...,...,...,...,...,...,...,...,...,...,...
83277,2022-12-28 22:33:08.442,FFF8009A67AF2B8DF2749D9C11C7DC86,fail,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885,12
83278,2022-12-28 22:33:31.650,FFF8009A67AF2B8DF2749D9C11C7DC86,complete,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885,12
83279,2022-12-28 22:34:01.408,FFF8009A67AF2B8DF2749D9C11C7DC86,start,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885,13
83280,2022-12-28 22:34:51.820,FFF8009A67AF2B8DF2749D9C11C7DC86,complete,A5DB077CF68CB173444D6AD8FC3E2BF1,2022-12-24 17:30:24.895,android,US,9.794331,3885,13


In [148]:
MasterDataset.to_csv('MasterDataset.csv')

In [145]:
MasterDataset.groupby('player_id')